# Data Acquisition via Web Scraping (Rotten Tomatoes)

---

## Phase 1: Setup and Request Preparation

### Import Modules

In [50]:
# Import all required Python modules for scraping and data handling
import numpy as np
import pandas as pd
import requests             # For submitting HTTP requests
import json
from bs4 import BeautifulSoup # The primary tool for parsing HTML

print("Modules imported.")

Modules imported.


### Obtain a User-agent String
We get a User-agent string, which is necessary to identify our client. Scrapers should always send this header, as many sites block requests without it.

In [51]:
# Use httpbin.org to get a standard User-agent string
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']

print(f"Obtained User-agent: {useragent}")

Obtained User-agent: python-requests/2.32.4


### Define Request Headers
We compile the User-agent and an identifying "From" email into the headers dictionary. It's polite scraping practice to include contact info.

In [52]:
# Define the headers dictionary
headers = {'User-agent': useragent,
          'From': 'gzg8pf@virginia.edu'} # Example 'atr8e@virginia.edu'

print("Headers dictionary defined.")

Headers dictionary defined.


### Define the URL and Submit the GET Request
This is the Extraction (E) phase of the web scrape. We submit the GET request to the target page.

In [53]:
# Define the target URL (Movies in Theaters page on Rotten Tomatoes)
url = "https://www.rottentomatoes.com/browse/movies_in_theaters/"

# Submit the request with the custom headers
r = requests.get(url, headers=headers)

# A <Response [200]> means the request was successful
print(f"Request submitted. Status: {r}")

Request submitted. Status: <Response [200]>


In [54]:
# Uncomment and run r.text below this if you want to see the raw output!
# Recomment and rerun to get rid of the raw output, it's way too much!
#r.text

### Commentary on the Request
The response object 'r' now contains the entire HTML source code of the webpage.

The next step in the scraping process is to parse this raw text so we can easily search for the movie data.

---

## Phase 2: HTML Parsing and Isolation

### Parse the HTML with BeautifulSoup
This is the start of the Transformation (T) phase. BeautifulSoup turns the raw HTML text into a navigable Python object.

In [55]:
# Parse the raw HTML text (r.text) using the built-in 'html.parser'
mysoup = BeautifulSoup(r.text, 'html.parser')

print("HTML content successfully parsed into a BeautifulSoup object.")

HTML content successfully parsed into a BeautifulSoup object.


In [56]:
# Uncomment and run mysoup below this if you want to see the parsed output!
# Recomment and rerun to get rid of the parsed output, it's way too much!
#mysoup

### Isolate All Movie Tiles
We use BeautifulSoup's find_all() method to locate the custom HTML tag used by Rotten Tomatoes to wrap each movie listing.

In [57]:
# Find all elements corresponding to a single movie tile
# The tag 'tile-dynamic' appears to wrap all the necessary data for one movie.
mylist = mysoup.find_all('tile-dynamic')

print(f"Isolated {len(mylist)} movie tiles found on the page.")

Isolated 38 movie tiles found on the page.


In [58]:
# Uncomment and run mylist below this if you want to see the selected output!
# Recomment and rerun to get rid of the selected output, it's way too much!
#mylist

### Isolate a Single Movie for Inspection
To determine how to extract the data, we must first inspect the structure of a single movie tile. We grab the first element ([0]) from our list.

In [59]:
# Assign the first movie tile element to the variable 'm' for inspection
m = mylist[0]

print("Isolated the first movie tile (m) for inspection.")
# Display the element 'm' (optional: uncomment 'm' to see the full code block)
m

Isolated the first movie tile (m) for inspection.


<tile-dynamic data-qa="tile">
<rt-img alt="One Battle After Another poster image" loading="lazy" slot="image" src="https://resizing.flixster.com/J4-IRAM-JCRt1httKD5HlGu4kF8=/206x305/v2/https://resizing.flixster.com/RsxXU0Hem_FDFjs0xcKd_kOwEfg=/fit-in/180x240/v2/https://resizing.flixster.com/yepsxuaCu3f7igZnTo2huAOVtCU=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzLzNhNmQzNDA0LWFhYjQtNDA5Mi04OTMyLTA2Y2U2OWI5ZjVmYS5qcGc="></rt-img>
<div data-track="scores" slot="caption">
<div class="score-wrap">
<score-icon-critics certified="" sentiment="positive" size="1"></score-icon-critics>
<rt-text class="critics-score" context="label" size="1">96%</rt-text>
</div>
<span class="p--small">One Battle After Another</span>
<span class="sr-only">Link to One Battle After Another</span>
</div>
</tile-dynamic>

### Extract the Movie Title
We search within the single tile (m) for the specific HTML element containing the title, using the tag (span) and its class name (p--small).

In [60]:
# Search within 'm' for the title: it's inside a <span> with the class 'p--small'
# We grab the first result [0] and its string content
title_example = m.find_all('span', 'p--small')[0].string

print(f"Example Title Extracted: {title_example}")

Example Title Extracted: One Battle After Another


### Cleaning the Data (Trimming the List)
Inspection of the page often reveals that the first few tiles are sometimes placeholders or ads. We trim the first 10 tiles to start with the main movie listings.

In [61]:
# Trim the first 10 elements which may be placeholder/ad content
mylist = mylist[10:]

print(f"Trimmed movie list to {len(mylist)} tiles.")

Trimmed movie list to 28 tiles.


In [62]:
# Uncomment and run mylist below this if you want to see the selected output!
# Recomment and rerun to get rid of the selected output, it's way too much!
#mylist

---

## Phase 3: Data Extraction

### Extract All Movie Titles
Now that we know the path to the title, we use a list comprehension to efficiently extract the title from every element in our cleaned mylist.

In [63]:
# List comprehension to extract and clean the title from every movie tile
# .text is more robust than .string, and .strip() removes leading/trailing whitespace
titles = [m.find_all('span', 'p--small')[0].text.strip() for m in mylist]

print(f"Extracted {len(titles)} movie titles.")
print("Example Titles:")
print(titles[:5]) # Display the first 5 titles

Extracted 28 movie titles.
Example Titles:
['Satisfied', 'Roads of Fire', "A Writer's Odyssey 2", 'The Smashing Machine', 'Anemone']


### Extract Score Attributes (Audience Example)
Scores often require checking specific HTML attributes. We examine one tile to find the HTML attribute (sentiment) used to store the score category.

In [64]:
# Isolate a specific tile for attribute checking
example_tile = mylist[0]

# The 'score-icon-critics' tag holds the sentiment in an attribute named 'sentiment'
critic_sentiment_example = example_tile.find_all('score-icon-critics')[0]['sentiment']

print(f"Example Critic Sentiment (Attribute): {critic_sentiment_example}")

Example Critic Sentiment (Attribute): positive


### Extract All Score Components
We repeat the list comprehension process for all required score components (Certified status, Score value, and Sentiment) for both Critics and Audience.

In [65]:
# Audience Data Extraction
audiencecertified = [m.find_all('score-icon-audience')[0]['certified'] for m in mylist]
audiencescore = [m.find_all('rt-text')[1].text.strip() for m in mylist]
audiencesentiment = [m.find_all('score-icon-audience')[0]['sentiment'] for m in mylist]

# Critics Data Extraction
criticscertified = [m.find_all('score-icon-critics')[0]['certified'] for m in mylist]
criticsscore = [m.find_all('rt-text')[0].text.strip() for m in mylist]
criticssentiment = [m.find_all('score-icon-critics')[0]['sentiment'] for m in mylist]

print("All six data lists (Scores, Sentiment, Certified status) successfully extracted.")

All six data lists (Scores, Sentiment, Certified status) successfully extracted.


---

## Phase 4: Data Consolidation and Export

### Create the Final DataFrame
We combine all the parallel lists into a single Pandas DataFrame.

In [66]:
rt_data = pd.DataFrame({
    'title': titles,
    'audience_certified': audiencecertified,
    'audience_score': audiencescore,
    'audience_sentiment': audiencesentiment,
    'critics_certified': criticscertified,
    'critics_score': criticsscore,
    'critics_sentiment': criticssentiment
})

print(f"DataFrame created with {len(rt_data)} rows.")

DataFrame created with 28 rows.


### View the Final DataFrame

In [67]:
# Display the resulting DataFrame
display(rt_data)

,title,audience_certified,audience_score,audience_sentiment,critics_certified,critics_score,critics_sentiment
0,Satisfied,,,,false,100%,positive
1,Roads of Fire,,,,false,,
2,A Writer's Odyssey 2,,,,false,,
3,The Smashing Machine,,,,true,76%,positive
4,Anemone,,,,false,62%,positive
5,Bone Lake,,,,false,94%,positive
6,Good Boy,,,,false,93%,positive
7,Avatar: The Way of Water,true,92%,positive,true,76%,positive
8,Casper,false,50%,negative,false,60%,positive
9,Taylor Swift | The Official Release Party of a...,,,,false,,


### Extract Movie `$\text{URL}$`s
For fun, we can extract the relative URL for each movie, which is nested under a different tag. This shows how quickly you could start to think about how you would develop a web crawler that could grab URLs and traverse a whole site!

In [68]:
# Find the <a> tag which contains the link, identified by its 'data-qa' attribute
mylist_links = mysoup.find_all('a', attrs = {'data-qa':"discovery-media-list-item-caption"})

In [69]:
# Extract the 'href' attribute and prepend the base URL
urls = ['https://www.rottentomatoes.com' + m['href'] for m in mylist_links]
urls

['https://www.rottentomatoes.com/m/satisfied',
 'https://www.rottentomatoes.com/m/roads_of_fire',
 'https://www.rottentomatoes.com/m/the_smashing_machine_2025',
 'https://www.rottentomatoes.com/m/anemone_2025',
 'https://www.rottentomatoes.com/m/bone_lake',
 'https://www.rottentomatoes.com/m/good_boy_2025',
 'https://www.rottentomatoes.com/m/avatar_the_way_of_water',
 'https://www.rottentomatoes.com/m/taylor_swift_the_official_release_party_of_a_showgirl',
 'https://www.rottentomatoes.com/m/coyotes_2025',
 'https://www.rottentomatoes.com/m/killing_faith',
 'https://www.rottentomatoes.com/m/scared_shitless',
 'https://www.rottentomatoes.com/m/shell_2024',
 'https://www.rottentomatoes.com/m/perfect_blue_1999',
 'https://www.rottentomatoes.com/m/orwell',
 'https://www.rottentomatoes.com/m/scurry',
 'https://www.rottentomatoes.com/m/one_big_happy_family',
 'https://www.rottentomatoes.com/m/soul_of_a_nation']

### Export to CSV
This is the final Load (L) phase.

In [70]:
# Define the output filename
output_filename = 'rotten_tomatoes_movies_in_theaters.csv'

# Export the DataFrame to a CSV file without the pandas index
rt_data.to_csv(output_filename, index=False)

print(f"--- Load Phase Complete ---")
print(f"Clean, scraped movie data successfully exported to: {output_filename}")

--- Load Phase Complete ---
Clean, scraped movie data successfully exported to: rotten_tomatoes_movies_in_theaters.csv
